In [1]:
#!/usr/bin/env python3
from argparse import ArgumentParser
from datetime import timedelta
from importlib import import_module
import logging.config
import os
from PIL import Image
from signal import SIGINT, SIGTERM
import sys
import time
import cv2
import json
import numpy as np
import tensorflow as tf
import common
import lbtoolbox as lb
import loss
from nets import NET_CHOICES
from heads import HEAD_CHOICES
#from tensorflow.contrib import slim

In [2]:
def ssim(img1, img2):
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1 ** 2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2 ** 2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map

In [5]:
# Applying the Operations such as Cartooning , Morphism , Bluring  , False Color , Pixelation , Sketching
score_cartooned = []
score_morphed = []
score_bluring = []
score_false_color = []
score_pixelated = []
score_sketching = []
score_ic=[]
#os.mkdir('content/transformed_img_cartooned')
#os.mkdir('content/transformed_img_morphed')
#os.mkdir('content/transformed_img_bluring')
#os.mkdir('content/transformed_img_false_color')
#os.mkdir('content/transformed_img_pixelation')
#os.mkdir('content/transformed_img_sketching')

args='image_root/Market-1501-v15.09.15'

for filename in os.listdir(args):
        if (filename == 'query_orig'):
            for imgname in os.listdir(args + '/' + filename):
                try:
                    img = cv2.imread(args + '/' + filename + '/' + imgname)
                    # converting the image into rgb format
                    colored = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    score_ic.append(ssim(img,colored))

                    # starting cartooning
                    cartooned = cv2.stylization(colored, sigma_s=100, sigma_r=0.20)
                    score_cartooned.append(ssim(colored, cartooned))
                    cv2.imwrite('content/transformed_img_cartooned/' + imgname, cartooned)

                    # applying the bluring to the image
                    blured = cv2.medianBlur(colored, 3)
                    score_bluring.append(ssim(colored, blured))
                    cv2.imwrite('content/transformed_img_bluring/' + imgname, blured)

                    # Pixelation Transformantion to the image
                    ht, wd = colored.shape[:2]
                    w, h = (64, 64)
                    temp = cv2.resize(colored, (w, h), interpolation=cv2.INTER_LINEAR)
                    pixelated = cv2.resize(temp, (wd, ht), interpolation=cv2.INTER_NEAREST)
                    score_pixelated.append(ssim(pixelated, colored))
                    cv2.imwrite('content/transformed_img_pixelation/' + imgname, pixelated)

                    # Applying the Morphology to the image
                    morphed = cv2.morphologyEx(colored, cv2.MORPH_OPEN, (3, 3))
                    score_morphed.append(ssim(colored, morphed))
                    cv2.imwrite('content/transformed_img_morphed/' + imgname, pixelated)

                    # Creating the Sketch of the image to the image
                    invert_img = cv2.bitwise_not(img)
                    blur_img = cv2.GaussianBlur(invert_img, (3, 3), 0)
                    invblur_img = cv2.bitwise_not(blur_img)
                    sketch_img = cv2.divide(img, invblur_img, scale=255.0)
                    score_sketching.append(ssim(img, sketch_img))
                    cv2.imwrite('content/transformed_img_sketching/' + imgname, sketch_img)

                    # Applying false color to the image
                    img_color = cv2.applyColorMap(colored, cv2.COLORMAP_JET)
                    score_false_color.append(ssim(colored, img_color))
                    cv2.imwrite('content/transformed_img_false_color/' + imgname, img_color)

                except:
                    continue



# print('..............................................')
print('Average SSIM Score IMG, Colored, I-C : ', np.mean(score_ic))
# print('..............................................')
                    
                    
# print('..............................................')
print('The Average SSIM Score of the Cartooned Images : ', np.mean(score_cartooned))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the Blured Images : ', np.mean(score_bluring))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the Morphed Images : ', np.mean(score_morphed))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the  Sketched Images : ', np.mean(score_sketching))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the False Colored Images : ', np.mean(score_false_color))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the Pixelated Images : ', np.mean(score_pixelated))
# print('..............................................')

Average SSIM Score IMG, Colored, I-C :  0.9520206845405728
The Average SSIM Score of the Cartooned Images :  0.18455576118828884
The Average SSIM Score of the Blured Images :  0.9263213760132635
The Average SSIM Score of the Morphed Images :  0.7941312377814084
The Average SSIM Score of the  Sketched Images :  0.38209486747244037
The Average SSIM Score of the False Colored Images :  0.14329781187716922
The Average SSIM Score of the Pixelated Images :  0.9290085841884319


In [4]:

# print('..............................................')
print('Average SSIM Score IMG, Colored, I-C : ', np.mean(score_ic))
# print('..............................................')
                    
                    
# print('..............................................')
print('The Average SSIM Score of the Cartooned Images : ', np.mean(score_cartooned))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the Blured Images : ', np.mean(score_bluring))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the Morphed Images : ', np.mean(score_morphed))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the  Sketched Images : ', np.mean(score_sketching))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the False Colored Images : ', np.mean(score_false_color))
# print('..............................................')

# print('..............................................')
print('The Average SSIM Score of the Pixelated Images : ', np.mean(score_pixelated))
# print('..............................................')

Average SSIM Score IMG, Colored, I-C :  0.9012169065974375
The Average SSIM Score of the Cartooned Images :  0.6439126346344022
The Average SSIM Score of the Blured Images :  0.9980320585937668
The Average SSIM Score of the Morphed Images :  0.9783597615029914
The Average SSIM Score of the  Sketched Images :  0.38636239765837604
The Average SSIM Score of the False Colored Images :  0.3938236244306261
The Average SSIM Score of the Pixelated Images :  0.9572409556326769
